# Determining the hours

In [1]:
# set up path to import my python scripts
import sys
import os
src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path+"/src")

In [2]:
%load_ext autoreload
%autoreload 2
import prep_data as prep

In [3]:
import pandas as pd
import numpy as np


In [4]:
# link for data files
base_url = "http://kopiko.ifa.hawaii.edu/weather/archivedata/"
# get list of all data file urls
csv_urls = prep.get_csv_file_links(base_url)

# data column names
column_names = ['date_time','temperature','pressure','humidity','wind_speed','wind_direction','visibility','co2','insolation','vertical_wind_speed','precipitation','10min','dewpoint']
columns_of_interest = ['date_time','temperature','humidity','wind_speed','visibility','precipitation','dewpoint','10min']

# Define reasonable ranges for each column
acceptable_ranges = {
    'temperature': (-273,40),
    'humidity': (0,100),
    'wind_speed': (0,100),
    'visibility': (0,100000),
    'precipitation': (0,100),
    'dewpoint': (-273,40)
    }
# Define the thresholds for ('Green', 'Red') weather - plan to use config file in future
thresholds = {
        'humidity': (75,85),
        'wind_sust': (10,12),
        'wind_gust': (15,15),
        'visibility': (50000,40000),
        'precipitation': (0,0),
        'dewpoint_delta': (6,3)
        }


In [5]:
#----TEMP----#
# for initial testing just grab one year - will  use a loop for this later
year = 2018
link = prep.get_specific_year(year,csv_urls)
df_2018 = prep.read_data_of_interest(link, column_names,columns_of_interest)

In [10]:
 # create a small subset of data for testing
df = df_2018.iloc[:100000].copy()

# check for reasonable values
prep.remove_unreasonable_measurements(df,acceptable_ranges,inplace=True)
# split wind into sustaind and gusts
df = prep.determine_wind_sust_and_gust(df)
# add delta dew point
df['dewpoint_delta'] = df['temperature'] - df['dewpoint']
# give other values for precipitation and visibility
# df['precipitation'] = np.random.binomial(1,.2,len(df))
# df.loc[np.random.choice(df.index.values,5),'visibility'] = 50000
# df.loc[np.random.choice(df.index.values,5),'visibility'] = 35000
# add status column
df['status'] = prep.get_weather_status(df,thresholds)
df

,temperature,humidity,wind_speed,visibility,precipitation,dewpoint,10min,wind_sust,wind_gust,dewpoint_delta,status
date_time,,,,,,,,,,,
2018-01-01 00:00:05,12.08,17.8,5.6,43682.0,NaN,-42.26,0,5.600000,5.6,54.34,Yellow
2018-01-01 00:00:15,12.09,18.1,5.3,43682.0,NaN,-46.06,0,5.450000,5.3,58.15,Yellow
2018-01-01 00:00:25,12.07,18.3,4.8,46022.0,NaN,-46.98,0,5.233333,4.8,59.05,Yellow
2018-01-01 00:00:36,12.07,18.3,3.9,48408.0,NaN,-38.20,0,4.900000,3.9,50.27,Yellow
2018-01-01 00:00:46,12.06,18.8,3.6,48408.0,NaN,-45.39,0,4.640000,3.6,57.45,Yellow
...,...,...,...,...,...,...,...,...,...,...,...
2018-01-12 23:25:33,11.81,57.7,5.3,49728.0,NaN,-48.86,0,3.433333,5.3,60.67,Yellow
2018-01-12 23:25:43,11.79,57.5,4.3,49633.0,NaN,-42.51,0,3.500000,4.3,54.30,Yellow
2018-01-12 23:25:54,11.80,57.7,4.4,49633.0,NaN,-41.70,0,3.566667,4.4,53.50,Yellow


In [12]:
# reset index so we can now grab dates
df = df.reset_index()
df

,date_time,temperature,humidity,wind_speed,visibility,precipitation,dewpoint,10min,wind_sust,wind_gust,dewpoint_delta,status
0,2018-01-01 00:00:05,12.08,17.8,5.6,43682.0,NaN,-42.26,0,5.600000,5.6,54.34,Yellow
1,2018-01-01 00:00:15,12.09,18.1,5.3,43682.0,NaN,-46.06,0,5.450000,5.3,58.15,Yellow
2,2018-01-01 00:00:25,12.07,18.3,4.8,46022.0,NaN,-46.98,0,5.233333,4.8,59.05,Yellow
3,2018-01-01 00:00:36,12.07,18.3,3.9,48408.0,NaN,-38.20,0,4.900000,3.9,50.27,Yellow
4,2018-01-01 00:00:46,12.06,18.8,3.6,48408.0,NaN,-45.39,0,4.640000,3.6,57.45,Yellow
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2018-01-12 23:25:33,11.81,57.7,5.3,49728.0,NaN,-48.86,0,3.433333,5.3,60.67,Yellow
99996,2018-01-12 23:25:43,11.79,57.5,4.3,49633.0,NaN,-42.51,0,3.500000,4.3,54.30,Yellow
99997,2018-01-12 23:25:54,11.80,57.7,4.4,49633.0,NaN,-41.70,0,3.566667,4.4,53.50,Yellow
99998,2018-01-12 23:26:04,11.80,57.7,3.1,50000.0,NaN,-46.89,0,3.500000,3.1,58.69,Yellow


In [18]:
df['status_change'] = df.status.shift() != df.status
df

,date_time,temperature,humidity,wind_speed,visibility,precipitation,dewpoint,10min,wind_sust,wind_gust,dewpoint_delta,status,status_chage,status_change
0,2018-01-01 00:00:05,12.08,17.8,5.6,43682.0,NaN,-42.26,0,5.600000,5.6,54.34,Yellow,True,True
1,2018-01-01 00:00:15,12.09,18.1,5.3,43682.0,NaN,-46.06,0,5.450000,5.3,58.15,Yellow,False,False
2,2018-01-01 00:00:25,12.07,18.3,4.8,46022.0,NaN,-46.98,0,5.233333,4.8,59.05,Yellow,False,False
3,2018-01-01 00:00:36,12.07,18.3,3.9,48408.0,NaN,-38.20,0,4.900000,3.9,50.27,Yellow,False,False
4,2018-01-01 00:00:46,12.06,18.8,3.6,48408.0,NaN,-45.39,0,4.640000,3.6,57.45,Yellow,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2018-01-12 23:25:33,11.81,57.7,5.3,49728.0,NaN,-48.86,0,3.433333,5.3,60.67,Yellow,False,False
99996,2018-01-12 23:25:43,11.79,57.5,4.3,49633.0,NaN,-42.51,0,3.500000,4.3,54.30,Yellow,False,False
99997,2018-01-12 23:25:54,11.80,57.7,4.4,49633.0,NaN,-41.70,0,3.566667,4.4,53.50,Yellow,False,False
99998,2018-01-12 23:26:04,11.80,57.7,3.1,50000.0,NaN,-46.89,0,3.500000,3.1,58.69,Yellow,False,False


In [19]:
sum(df.status_change)

9615

In [27]:
df.loc[df.status_change].index

Int64Index([    0,   153,   156,   285,   286,   292,   293,   351,   353,
              392,
            ...
            99855, 99865, 99927, 99932, 99953, 99965, 99970, 99974, 99980,
            99984],
           dtype='int64', length=9615)

In [28]:
df.index[df.status_change]

Int64Index([    0,   153,   156,   285,   286,   292,   293,   351,   353,
              392,
            ...
            99855, 99865, 99927, 99932, 99953, 99965, 99970, 99974, 99980,
            99984],
           dtype='int64', length=9615)